In [1]:
import tensorflow as tf
print(tf.__version__)

2.10.0


In [2]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import gym

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
env = gym.make('CartPole-v0')

num_actions = env.action_space.n
print("num actions : ",num_actions)

state_shape = env.observation_space.shape[0]
print("state shape : ",state_shape)

gamma = 0.95

num actions :  2
state shape :  4


In [4]:
def discount_and_normalize_rewards(episode_rewards):
    discounted_rewards = np.zeros_like(episode_rewards)
    
    reward_to_go = 0.0
    for i in reversed(len(episode_rewards)):
        reward_to_go = reward_to_go * gamma * episode_rewards[i]
        discounted_rewards[i] = reward_to_go
    
    # normalize and return the reward
    discounted_rewards -= np.mean(discounted_rewards)
    discounted_rewards /= np.std(discounted_rewards)
    
    return discounted_rewards

# Building the policy network

In [5]:
tf.reset_default_graph()

In [6]:
state_ph = tf.placeholder(tf.float32, [None, state_shape], name="state_ph")

action_ph = tf.placeholder(tf.float32, [None, num_actions], name="action_ph")

discounted_rewards_ph = tf.placeholder(tf.float32, [None,], name="discounted_rewards")

## layer

In [7]:
layer1 = tf.layers.dense(state_ph, units=32, activation=tf.nn.relu)

layer2 = tf.layers.dense(layer1, units=num_actions)

prob_dist = tf.nn.softmax(layer2)

We learned that we compute gradient as:

$$\nabla_{\theta} J(\theta) = \frac{1}{N} \sum_{i=1}^{N}\left[\sum_{t=0}^{T-1}  \nabla_{\theta} \log \pi_{\theta}\left(a_{t} | s_{t}\right)R_t\right] $$
    
After computing the gradient we update the parameter of the network using the gradient
ascent as:    

$$\theta = \theta + \alpha \nabla_{\theta} J(\theta) $$

However, it is a standard convention to perform minimization rather than maximization.
So, we can convert the above maximization objective into the minimization objective by just
adding a negative sign.

Thus, we can define negative log policy as:


In [9]:
neg_log_policy = tf.nn.softmax_cross_entropy_with_logits_v2(logits = layer2, labels = action_ph)

loss = tf.reduce_mean(neg_log_policy * discounted_rewards_ph)

train = tf.train.AdamOptimizer(0.01).minimize(loss)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    


with tf.Session() as sess:
    writer = tf.summary.FileWriter("./graphs", sess.graph)
    print(sess.run(train))
    

InvalidArgumentError: Graph execution error:

Detected at node 'action_ph' defined at (most recent call last):
    File "c:\Users\cc\miniconda3\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\cc\miniconda3\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "c:\Users\cc\miniconda3\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Users\cc\miniconda3\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
      app.start()
    File "c:\Users\cc\miniconda3\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "c:\Users\cc\miniconda3\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "c:\Users\cc\miniconda3\lib\asyncio\base_events.py", line 603, in run_forever
      self._run_once()
    File "c:\Users\cc\minicon

In [ ]:
d p

## Training the network

In [ ]:
#num_iterations = 1000
num_iterations = 1


with tf.Session() as sess:
    
    # initialize all the TensorFlow variables
    sess.run(tf.gloabal_variables_initializer())
    
    # for every iteration
    for i in range(num_iterations):
        # initialize an empty list for storing the states, actions, and rewards obtained in the episode
        episode_states, episode_actions, episode_rewards = [], [], []
        
        # set the done to False
        done = False
         
        state = env.reset()
        Return = 0
        
        while not done:
            # reshape the state
            state = state.reshape([1, 4])
            
            pi = sess.run(prob_dist, feed_dict = {state_ph : state})
            
            a = np.random.choice(range(pi.shape[1]), p=pi.ravel())
            
            next_state, reward, done, info = env.step(a)
            
            env.render()
            
            Return += reward
            
            action = np.zeros(num_actions)
            action[a] = 1
            
            # Store the state, action, and reward into their respective list
            episode_states.append(state)
            episode_states.append(action)
            episode_states.append(reward)
            
            # update the state to the next state
            state = next_state
            
        # compute the discounted and normalized reward
        discounted_rewards = discount_and_normalize_rewards(episode_rewards)
        
        # define the feed dictionary
        feed_dict = {state_ph : np.vstack(np.array(episode_states)),
                     action_ph : np.vstack(np.array(episode_actions)),
                     discounted_rewards_ph : discounted_rewards
                     }
        
        # train the network
        loss, _ = sess.run([loss, train], feed_dict = feed_dict)
        
        # print the return for every 10 iteration
        if i%10 == 0:
            print("Iteration : {}, Return : {}".format(i, Return))


with tf.Session() as sess:
    writer = tf.summary.FileWriter(logdir="./graphs", graph = sess.graph)
    print(sess.run(sum))
            